In [1]:
import os
from datasets import load_dataset, load_metric
import shutil
import numpy as np


import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer,  DataCollatorForTokenClassification, EarlyStoppingCallback
import transformers


#local directories

from pathlib import Path
BASE_DIR = Path().cwd().parent
wikiann_dl_dir = BASE_DIR.joinpath("data/interim")

## Задаем параметр модели

In [2]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "DeepPavlov/rubert-base-cased"#"bert-base-multilingual-uncased"#"mrm8488/distilbert-base-multi-cased-finetuned-typo-detection" "distilbert-base-uncased" 
batch_size = 32
# batch_size = 32

## Подгружаем данные

In [3]:
shutil.rmtree(str(wikiann_dl_dir.joinpath('cache')))

os.mkdir(str(wikiann_dl_dir.joinpath('cache')))

datasets = load_dataset(str(wikiann_dl_dir.joinpath("wikiann_local.py")), data_files=str(wikiann_dl_dir.joinpath("default.zip")), 
                        cache_dir=str(wikiann_dl_dir.joinpath('cache')))

Using custom data configuration default-c7de00f0f2af4a11


Dataset wikiann downloaded and prepared to /home/ewp/DEV/ld_ner_bert_learn/data/interim/cache/wikiann/default-c7de00f0f2af4a11/1.1.0/5695de592d40d13b53e20479dbfaa3948fc461d991d92cb4ae429105e4d5d170. Subsequent calls will reuse this data.


In [4]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

['O',
 'B-PER',
 'I-PER',
 'B-ORG',
 'I-ORG',
 'B-LOC',
 'I-LOC',
 'B-INNKPP',
 'I-INNKPP',
 'B-RSKS',
 'I-RSKS',
 'B-STAT',
 'I-STAT']

In [5]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))
    
show_random_elements(datasets["train"])

,langs,ner_tags,spans,tokens
0,"[ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru]","[B-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, O, O, O, O, O, O]",[ORG: АО 79 ЦЕНТРАЛЬНАЯ ИНЖЕНЕРНАЯ БАЗА Филиал],"[АО, 79, ЦЕНТРАЛЬНАЯ, ИНЖЕНЕРНАЯ, БАЗА, Филиал, именуемое, дальнейшем, ""Подрядчик"", с, одной, стороны]"
1,"[ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru]","[O, B-ORG, I-ORG, I-ORG, I-ORG, O, B-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, O, O, B-INNKPP, O, O, O]","[ORG: "" МИА-ФАРМ "" ООО, ORG: Коммерческий Банк "" Кутузовский "" Общество с ограниченной ответственностью, INNKPP: 8613007049]","[Исполнитель, "", МИА-ФАРМ, "", ООО, рс, Коммерческий, Банк, "", Кутузовский, "", Общество, с, ограниченной, ответственностью, ИНН, КПП, 8613007049, /, тел., к/счет]"
2,"[ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru]","[O, B-ORG, I-ORG, B-PER, I-PER, I-PER, O, B-INNKPP, O, O, O, O, B-LOC, I-LOC, I-LOC, I-LOC, I-LOC, I-LOC, I-LOC, I-LOC, I-LOC, I-LOC, I-LOC, I-LOC, I-LOC, O, O, B-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, O, B-STAT, I-STAT, I-STAT, I-STAT, I-STAT, I-STAT]","[ORG: Индивидуальный предприниматель, PER: Ысакова Бурулкан Нурдиновна, INNKPP: 7704845575, LOC: 654222 , Кемеровская область - Кузбасс , Новокузнецкий р-н , поселок Чистая Грива, ORG: Акционерный коммерческий банк "" Инвестбанк "" ( открытое акционерное общество ), STAT: 8 ( 843 ) 723 1911]","[Заказчик, Индивидуальный, предприниматель, Ысакова, Бурулкан, Нурдиновна, ИНН, 7704845575, Адрес, филиала, банка, Клиент, 654222, ,, Кемеровская, область, -, Кузбасс, ,, Новокузнецкий, р-н, ,, поселок, Чистая, Грива, Филиал, банка, Акционерный, коммерческий, банк, "", Инвестбанк, "", (, открытое, акционерное, общество, ), корр.сч, 8, (, 843, ), 723, 1911]"
3,"[ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru]","[O, B-ORG, I-ORG, B-PER, I-PER, I-PER, O, O, O, O, B-INNKPP, O, O, O, B-LOC, I-LOC, I-LOC, I-LOC, I-LOC, I-LOC, I-LOC, I-LOC, I-LOC, O, O, B-STAT, O]","[ORG: Индивидуальный предприниматель, PER: Чужинина Вера Николаевна, INNKPP: 431201001, LOC: 654000 , Кемеровская область - Кузбасс , г Новокузнецк, STAT: 041161321]","[Лицензиат, Индивидуальный, предприниматель, Чужинина, Вера, Николаевна, ИНН, КПП, Поставщик, /, 431201001, Адрес, филиала, банка, 654000, ,, Кемеровская, область, -, Кузбасс, ,, г, Новокузнецк, БИК, Банка, 041161321, тел.]"
4,"[ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru, ru]","[O, B-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, O, O, O, B-INNKPP, O, O, B-LOC, I-LOC, I-LOC, I-LOC, I-LOC, I-LOC, I-LOC, I-LOC, O, O, B-LOC, I-LOC, I-LOC, I-LOC, I-LOC, I-LOC, I-LOC, I-LOC, O, O, O, B-ORG, I-ORG, I-ORG, I-ORG, I-ORG, O, O, B-STAT, O, B-RSKS, O, B-RSKS]","[ORG: ВОЛГОГРАДСКИЙ ФИЛИАЛ АКЦИОНЕРНОГО ОБЩЕСТВА "" СТРАХОВОЕ ОБЩЕСТВО ГАЗОВОЙ ПРОМЫШЛЕННОСТИ "", INNKPP: 7802553644, LOC: ОБЛАСТЬ РОСТОВСКАЯ ГОРОД РОСТОВ-НА-ДОНУ УЛИЦА БОЛЬШАЯ САДОВАЯ 188А, LOC: ОБЛАСТЬ НОВОСИБИРСКАЯ ГОРОД НОВОСИБИРСК УЛИЦА ДОБРОЛЮБОВА ДОМ 160, ORG: АКЦИОНЕРНОЕ ОБЩЕСТВО "" ВЛАДБИЗНЕСБАНК "", STAT: 049090650, RSKS: 40712478033062647294, RSKS: 30200193850590030083]","[Подрядчик, ВОЛГОГРАДСКИЙ, ФИЛИАЛ, АКЦИОНЕРНОГО, ОБЩЕСТВА, "", СТРАХОВОЕ, ОБЩЕСТВО, ГАЗОВОЙ, ПРОМЫШЛЕННОСТИ, "", тел., ИНН, Субподрядчик, 7802553644, Адрес, банка, ОБЛАСТЬ, РОСТОВСКАЯ, ГОРОД, РОСТОВ-НА-ДОНУ, УЛИЦА, БОЛЬШАЯ, САДОВАЯ, 188А, Адрес, Исполнитель, ОБЛАСТЬ, НОВОСИБИРСКАЯ, ГОРОД, НОВОСИБИРСК, УЛИЦА, ДОБРОЛЮБОВА, ДОМ, 160, тел., Филиал, банка, АКЦИОНЕРНОЕ, ОБЩЕСТВО, "", ВЛАДБИЗНЕСБАНК, "", БИК, Банка, 049090650, рс.сч., 40712478033062647294, кор/счет, 30200193850590030083]"
5,"[ru, ru, ru, ru

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, do_lower_case=True)
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [7]:
tokenizer("Привет, это одно предложение!!!")

{'input_ids': [101, 26856, 128, 3998, 7635, 16541, 106, 106, 106, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [7]:
example = datasets["train"][3]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', '"', 'ва', '##б', '##ко', 'рус', '"', 'о', '##оо', 'адрес', 'р', '.', 'сч', '.', '433', '##28', '##47', '##43', '##24', '##56', '##19', '##67', '##82', '##9', 'банк', 'исп', '-', 'тель', '/', '352', '##801', '##001', 'тел', '.', '+', '7', '(', '835', ')', '03', '74', '505', 'бик', '048', '##56', '##78', '##63', '[SEP]']


In [8]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

48 48


In [9]:
label_all_tokens = True

In [10]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [11]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [12]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [14]:
metric = load_metric("seqeval")

labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'INNKPP': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'RSKS': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'STAT': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [15]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
# Замораживаем все параметры, кроме головы
# for param in model.distilbert.parameters():
for param in model.bert.parameters():
    param.requires_grad = False

In [ ]:
for param in model.parameters():
    if param.requires_grad:
        print(param)

In [ ]:
# разморозка - не запускаем
for param in model.parameters():
    param.requires_grad = True

In [18]:
args = TrainingArguments(
    f"test-{task}",
    overwrite_output_dir=True,
    evaluation_strategy = "epoch",
    learning_rate=2e-4, #2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.001, #0.01,
    save_strategy="epoch",
    
)



trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

In [ ]:
# Разовый запуск обучения

trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

In [ ]:
trainer.save_model(BASE_DIR.joinpath('models/ner_dpbert_37epoch_86f1_93acc.bin'))

## Подбор параметров

In [16]:

def get_tainer(lgrid, model, tokenized_datasets, compute_metrics, data_collator):
    args = TrainingArguments(
        f"test-{task}",
        overwrite_output_dir=True,
        evaluation_strategy = "epoch",
        learning_rate=lgrid['learning_rate'],
        per_device_train_batch_size=lgrid['batch_size'],
        per_device_eval_batch_size=lgrid['batch_size'],
        num_train_epochs=30,
        weight_decay=lgrid['weight_decay'],
        save_strategy="no",
        load_best_model_at_end = False,
    )

    # Замораживаем все параметры, кроме головы
    # for param in model.distilbert.parameters():
    for param in model.bert.parameters():
        param.requires_grad = False

    return Trainer(
                    model,
                    args,
                    train_dataset=tokenized_datasets["train"],
                    eval_dataset=tokenized_datasets["validation"],
                    data_collator=data_collator,
                    tokenizer=tokenizer,
                    compute_metrics=compute_metrics,
                    
#                     callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
                )

In [20]:
# проверяем работоспособность процедуры
i = {'learning_rate': 2e-05, 'batch_size': 16, 'weight_decay': 0.0}
get_tainer(i, model, tokenized_datasets, compute_metrics, data_collator)

In [17]:
# создаем сетку параметров
grid = {'learning_rate': [5e-5, 2e-4, 2e-3],
        'batch_size': [32],
       'weight_decay':[0.0, 0.1, 0.01]}

In [18]:
from itertools import product

def process(d):
    to_product = []  # [[('a', 1), ('a', 2)], [('b', 3),], ...]
    for k, v in d.items():
        if isinstance(v, list):
            to_product.append([(k, i) for i in v])
        elif isinstance(v, dict):
            to_product.append([(k, i) for i in process(v)])
        else:
            to_product.append([(k, v)])
    return [dict(l) for l in product(*to_product)]

In [19]:
# пробегаемся по сетке, фиксируем результаты обучения в массивчик
results = []
for i in process(grid)[:]:
    print(i)
    model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
    trainer = get_tainer(i, model, tokenized_datasets, compute_metrics, data_collator)
    trainer.train()
    
    
    predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    rr = metric.compute(predictions=true_predictions, references=true_labels)
    results.append([i, rr])
    print(i, ' = ', rr['ORG'])

{'learning_rate': 5e-05, 'batch_size': 32, 'weight_decay': 0.0}


Some weights of BertForTokenClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.561983,0.368171,0.484585,0.418432,0.611974
2,No log,1.118332,0.451447,0.534347,0.489411,0.701939
3,No log,0.919972,0.500901,0.567521,0.532134,0.739369
4,1.485300,0.803444,0.546136,0.601129,0.572314,0.768072
5,1.485300,0.725399,0.577852,0.634433,0.604823,0.788184
6,1.485300,0.667748,0.603770,0.657881,0.629665,0.803365
7,0.836000,0.623253,0.624749,0.675423,0.649099,0.814437
8,0.836000,0.588452,0.637256,0.687451,0.661403,0.822237
9,0.836000,0.559766,0.650601,0.699870,0.674337,0.829483
10,0.676500,0.535807,0.661009,0.711984,0.685551,0.835251


/home/ewp/.local/lib/python3.7/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'learning_rate': 5e-05, 'batch_size': 32, 'weight_decay': 0.0}  =  {'precision': 0.3459698196540302, 'recall': 0.4117389399912396, 'f1': 0.376, 'number': 2283}
{'learning_rate': 5e-05, 'batch_size': 32, 'weight_decay': 0.1}


...

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.291966,0.772522,0.817673,0.794456,0.901533
2,No log,0.235690,0.806754,0.843291,0.824618,0.919852
3,No log,0.209906,0.824733,0.859184,0.841606,0.928742
4,0.401400,0.202060,0.828342,0.864742,0.846151,0.930042
5,0.401400,0.189535,0.839590,0.877942,0.858338,0.936303
6,0.401400,0.185921,0.842172,0.869561,0.855647,0.935840
7,0.279600,0.182935,0.841862,0.877942,0.859523,0.937438
8,0.279600,0.176660,0.848600,0.879331,0.863693,0.940606
9,0.279600,0.174496,0.851271,0.881285,0.866018,0.940397
10,0.267800,0.172401,0.855349,0.884542,0.869701,0.941219


{'learning_rate': 0.002, 'batch_size': 32, 'weight_decay': 0.1}  =  {'precision': 0.5260070394994134, 'recall': 0.5891371003066141, 'f1': 0.5557851239669421, 'number': 2283}
{'learning_rate': 0.002, 'batch_size': 32, 'weight_decay': 0.01}


Some weights of BertForTokenClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.290302,0.772669,0.818063,0.794719,0.901742
2,No log,0.233114,0.807718,0.844333,0.825620,0.920345
3,No log,0.206741,0.826076,0.860834,0.843097,0.929519
4,0.399400,0.198413,0.830814,0.866348,0.848209,0.931267
5,0.399400,0.185554,0.841312,0.879852,0.860151,0.937304
6,0.399400,0.181788,0.843683,0.871342,0.857289,0.936766
7,0.276600,0.177940,0.844592,0.881155,0.862486,0.938977
8,0.276600,0.171937,0.850507,0.881676,0.865811,0.941801
9,0.276600,0.169465,0.854124,0.884498,0.869046,0.942115
10,0.264700,0.166989,0.857629,0.886974,0.872054,0.942877


{'learning_rate': 0.002, 'batch_size': 32, 'weight_decay': 0.01}  =  {'precision': 0.530373831775701, 'recall': 0.5965834428383706, 'f1': 0.5615337043908473, 'number': 2283}


In [ ]:
# Просмотр результатов подбора параметров


rr = []
for i in results:
    p = i[1]['ORG']
    p['params'] = str(i[0])
    rr.append(p)
df_f1 = pd.DataFrame(rr).sort_values(by=['f1'])

display(HTML(df_f1.to_html()))

,precision,recall,f1,number,params
1,0.344359,0.410425,0.374500,2283,"{'learning_rate': 5e-05, 'batch_size': 32, 'weight_decay': 0.1}"
2,0.344726,0.410863,0.374900,2283,"{'learning_rate': 5e-05, 'batch_size': 32, 'weight_decay': 0.01}"
0,0.345970,0.411739,0.376000,2283,"{'learning_rate': 5e-05, 'batch_size': 32, 'weight_decay': 0.0}"
4,0.451429,0.519054,0.482885,2283,"{'learning_rate': 0.0002, 'batch_size': 32, 'weight_decay': 0.1}"
3,0.451502,0.519930,0.483306,2283,"{'learning_rate': 0.0002, 'batch_size': 32, 'weight_decay': 0.0}"
5,0.451674,0.519930,0.483405,2283,"{'learning_rate': 0.0002, 'batch_size': 32, 'weight_decay': 0.01}"
7,0.526007,0.589137,0.555785,2283,"{'learning_rate': 0.002, 'batch_size': 32, 'weight_decay': 0.1}"
8,0.530374,0.596583,0.561534,2283,"{'learning_rate': 0.002, 'batch_size': 32, 'weight_decay': 0.01}"
6,0.531128,0.597898,0.562539,2283,"{'learning_rate': 0.002, 'batch_size': 32, 'weight_decay': 0.0}"
